In [1]:
from transformers import AutoModelForSequenceClassification,RobertaTokenizer, Trainer, TrainingArguments
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
import numpy as np
import os

c:\Users\tonya\OneDrive\Desktop\ML_Final\MachineLearning-Final-Project-Files\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.label = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
       
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation = True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding =  'max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
   


        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'label': torch.tensor(self.label[index], dtype=torch.float)
        }

In [3]:

def encode_Lables(label):
    temp = np.zeros(5)
    temp[label-1] = 1
    return temp


    
def split_data(filename,samp_size):
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    path = parent_dir +"/model_data/" + filename
    #Sampling data
    if samp_size > 1:
        samp_size = 1
    if samp_size <=0:
      samp_size = .1

    df = pd.read_csv(path)  
    df = df.sample(n=int(len(df) * samp_size))
    df.reset_index(inplace=True)
    df = pd.read_csv(path)
    df = df.sample(n=int(len(df) * .50), replace=True)

    #Renaming for Model
    df.rename(columns={"Rating":"label"},inplace=True)
    df.rename(columns={"Review_text":"text"},inplace=True)
    df["label"] = df["label"].apply(encode_Lables)
    train_data,test_data = train_test_split(df,test_size=.20,random_state=64)
    train_data = train_data.reset_index(drop=True)
    test_data = test_data.reset_index(drop=True)
    max_length = int(df["text"].str.split().str.len().mean())
    return train_data,test_data,max_length



In [4]:
#hyperParmaters
samp_size = .3
lr = 2e-5
epochs = 5
train_batch_size = 32
test_batch_size = 32
w_decay = .001

In [5]:

filename = "data_set_1.csv"

train,test,max_length= split_data(filename,samp_size)

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=5)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_set = SentimentData(train, tokenizer, max_len=max_length)
testing_set = SentimentData(test, tokenizer, max_len=max_length)

In [8]:
training_set.__getitem__(5)

{'input_ids': tensor([   0, 5715, 2369, 1318,    2,    1,    1,    1,    1,    1,    1,    1,
            1,    1]),
 'mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'label': tensor([0., 0., 0., 0., 1.])}

In [9]:
def compute_metrics(pred):
    
    label = torch.argmax(torch.tensor(pred.label_ids),dim=1).numpy()
    preds = torch.argmax(torch.tensor(pred.predictions),dim=1).numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(label, preds, average='macro')
    acc = accuracy_score(label, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [10]:
training_args = TrainingArguments(

    learning_rate=lr,
    fp16=True,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=test_batch_size,
    num_train_epochs=epochs,
    weight_decay=w_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_set,
    eval_dataset=testing_set,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.355600,0.310786,0.646552,0.312974,0.355019,0.362655
2,0.308100,0.301704,0.655172,0.368049,0.387143,0.394118
3,0.289900,0.301195,0.651525,0.397269,0.398883,0.412480
4,0.269000,0.299741,0.656001,0.417397,0.416876,0.424576
5,0.257000,0.302143,0.658488,0.436206,0.505987,0.434711


c:\Users\tonya\OneDrive\Desktop\ML_Final\MachineLearning-Final-Project-Files\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tonya\OneDrive\Desktop\ML_Final\MachineLearning-Final-Project-Files\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tonya\OneDrive\Desktop\ML_Final\MachineLearning-Final-Project-Files\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samp

TrainOutput(global_step=3770, training_loss=0.2933345703610691, metrics={'train_runtime': 546.1784, 'train_samples_per_second': 220.853, 'train_steps_per_second': 6.903, 'total_flos': 867853052902500.0, 'train_loss': 0.2933345703610691, 'epoch': 5.0})